In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        'notebook { padding-top:0px !important; } '
        'notebook { padding-bottom:0px !important; } '
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

/tmp/ipykernel_10231/2869641199.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import pandas as pd

import random

from itertools import permutations

In [3]:
teacher_max = pd.read_excel('teachers_max_hours.xlsx')

print(teacher_max.groupby('teacher').h_max.sum())
print(teacher_max.groupby('classroom').h_max.sum())

# check on N
if teacher_max.teacher.nunique() < teacher_max.classroom.nunique():
    print('N teachers cannot be smaller than N classroms!')
    
teacher_max

teacher
andrea        12
antonietta    16
maria         13
paola         19
Name: h_max, dtype: int64
classroom
1    20
2    20
3    20
Name: h_max, dtype: int64


,teacher,h_max,classroom
0,antonietta,3,1
1,paola,8,1
2,maria,6,1
3,antonietta,4,2
4,paola,9,2
5,maria,3,2
6,antonietta,9,3
7,paola,2,3
8,maria,4,3
9,andrea,3,1


In [4]:
days = ['mon', 'tue', 'wed', 'thu' , 'fri'] 

## brute force approach

###  weak-contraints 
check only total h, **per teacher**

In [6]:
%%time

c=0

perf_dict = {}
# schedule = {}

while c < 10**4:

    # check N h of each teacher is respected
#     while all([str(schedule).count(t) == teacher_max[teacher_max.teacher==t].h_max.sum() for t in teacher_max.teacher.unique()])==False:
    schedule = {}
    c+=1

    for day in days:
        schedule[day] = {}

        for h in range(1,5):
            schedule[day][h] = random.choice(list(permutations(teacher_max.teacher.unique(),  teacher_max.classroom.nunique())))
            
    perf = sum(abs(str(schedule).count(t) - teacher_max[teacher_max.teacher==t].h_max.sum()) for t in teacher_max.teacher.unique())
    
    if perf == 0:
        
        perf_dict[c] = {}
        perf_dict[c]['perf'] = perf
        perf_dict[c]['solution'] = schedule

print('number of solutions found: ',len(perf_dict))

CPU times: user 30.8 s, sys: 15.8 ms, total: 30.9 s
Wall time: 30.8 s


In [10]:
perf_dict[614]['solution']

{'mon': {1: ('andrea', 'paola', 'maria'),
  2: ('andrea', 'antonietta', 'paola'),
  3: ('andrea', 'paola', 'antonietta'),
  4: ('antonietta', 'paola', 'maria')},
 'tue': {1: ('andrea', 'maria', 'antonietta'),
  2: ('paola', 'maria', 'andrea'),
  3: ('paola', 'antonietta', 'andrea'),
  4: ('maria', 'paola', 'antonietta')},
 'wed': {1: ('andrea', 'maria', 'paola'),
  2: ('andrea', 'antonietta', 'paola'),
  3: ('paola', 'antonietta', 'maria'),
  4: ('andrea', 'antonietta', 'paola')},
 'thu': {1: ('antonietta', 'paola', 'maria'),
  2: ('antonietta', 'maria', 'paola'),
  3: ('maria', 'antonietta', 'paola'),
  4: ('paola', 'maria', 'antonietta')},
 'fri': {1: ('maria', 'antonietta', 'paola'),
  2: ('paola', 'andrea', 'antonietta'),
  3: ('antonietta', 'paola', 'andrea'),
  4: ('andrea', 'paola', 'maria')}}

### strong-constraint 

checking total h , **per teacher and classrom**

In [11]:
from collections import Counter

In [57]:
%%time

c=0

perf_dict = {}
# schedule = {}

while c < 10**4:

    # check N h of each teacher is respected
#     while all([str(schedule).count(t) == teacher_max[teacher_max.teacher==t].h_max.sum() for t in teacher_max.teacher.unique()])==False:
    schedule = {}
    c+=1

    for day in days:
        schedule[day] = {}

        for h in range(1,5):
            schedule[day][h] = random.choice(list(permutations(teacher_max.teacher.unique(),  
                                                               teacher_max.classroom.nunique())))

    flattened = pd.json_normalize(schedule).to_dict(orient='records')[0]
    
    perf = {}
    for r in teacher_max.classroom.unique():
        class_count = Counter([v[r-1] for v in flattened.values()])
        perf[r] = all([teacher_max[(teacher_max.teacher==t) & (teacher_max.classroom==r)].h_max.values[0] == v 
                       for t,v in class_count.items()])
        
#     if all(perf.values()):
    if any(perf.values()): # try to look for at least 1 correct classrom
         
        perf_dict[c] = {}
        perf_dict[c]['perf'] = perf
        perf_dict[c]['solution'] = schedule
        
# perf_dict

CPU times: user 1min 26s, sys: 252 ms, total: 1min 26s
Wall time: 1min 26s


{18: {'perf': {1: False, 2: True, 3: False},
  'solution': {'mon': {1: ('antonietta', 'paola', 'maria'),
    2: ('andrea', 'paola', 'antonietta'),
    3: ('andrea', 'paola', 'maria'),
    4: ('andrea', 'maria', 'antonietta')},
   'tue': {1: ('maria', 'paola', 'andrea'),
    2: ('antonietta', 'andrea', 'paola'),
    3: ('paola', 'maria', 'andrea'),
    4: ('andrea', 'antonietta', 'paola')},
   'wed': {1: ('paola', 'maria', 'andrea'),
    2: ('andrea', 'antonietta', 'maria'),
    3: ('antonietta', 'paola', 'andrea'),
    4: ('antonietta', 'paola', 'andrea')},
   'thu': {1: ('maria', 'antonietta', 'andrea'),
    2: ('maria', 'paola', 'antonietta'),
    3: ('paola', 'antonietta', 'maria'),
    4: ('antonietta', 'paola', 'maria')},
   'fri': {1: ('antonietta', 'andrea', 'maria'),
    2: ('andrea', 'paola', 'maria'),
    3: ('paola', 'andrea', 'antonietta'),
    4: ('antonietta', 'andrea', 'maria')}}},
 347: {'perf': {1: False, 2: True, 3: False},
  'solution': {'mon': {1: ('maria', 'paola',

in the best case we get 1 classroom right

In [64]:
# [v['perf'] for v in perf_dict.values()]